<center> <h1>PRIMERA PREDICIÓN DEL SUEÑO DE UN PACIENTE</h1> </center>
<center> <h4>Noemi González, Roberto Holgado y Carmen Plaza seco</h4> </center>

[URL PHYSIONET](https://physionet.org/content/sleep-accel/1.0.0/)

###### Indice

* **1. Preprocesado, extracción de características.**
  *   **1.1 Extracción variables de frecuencia cardíaca**
  *  **1.2 Extracción variables de movimiento**
  *  **1.3 Obtención matrices**
* **2. Conjuntos Train y Test**
  * **2.1 Preparación datos de entreada y salida**
* **3. Clasificador SVM Binario**
   * **3.1 Creación del modelo y selección parámetros libres**
   * **3.2 Evaluación del modelo**
* **4. Clasficador SVM Multiclase**
   * **4.1 Creación del modelo y selección parámetros libres**
      *     **4.2.1**
      *     **4.2.2**
   * **4.3 Evaluación del modelo**
* **5. Clasificador multiclase primera ventana**
    * **5.1 Residuos**

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import statistics as stats
from sklearn.model_selection import GridSearchCV
from random import sample
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV

## 1.Preprocesado, extracción de características

In [2]:
pacientes_sueño=[]
for file in (os.listdir("labels")):
    pacientes_sueño.append(os.path.join("labels",file))
    pacientes_sueño=sorted(pacientes_sueño)
    
pacientes_frec_card=[]
for file in os.listdir("heart_rate"):
    pacientes_frec_card.append(os.path.join("heart_rate",file))

pacientes_pasos=[]
for file in os.listdir("steps"):
    pacientes_pasos.append(os.path.join("steps",file))

pacientes_movimiento=[]
for file in os.listdir("motion"):
    pacientes_movimiento.append(os.path.join("motion",file))

La siguiente celda de código se corresponde con la ventana deslizante que recorrerá la señal fisiológica deseada. La función toma como datos de entrada el tamaño de la ventana en segundos y la señal sobre la que se quiere trabajar.

### 1.1 Extracción variables de frecuencia cardíaca

In [3]:
def get_window(w_size,signal,n_sueño):
    time = signal["Tiempo"]
    f_list=[]
    
    for i in range(n_sueño):
        a = w_size*(1+i)
        b = w_size*(2+i)

        idx=[]
        for t in time:
            if t>a and t<b:
                idx.append(True)
            else:
                idx.append(False)
        
        f = signal[idx]
        f_list.append(f)
        
    return f_list

En todos los casos, se seleccionará una w_size de 30 segundos para tener una característica cada 30s y así coincidir la cantidad de los datos adquiridos por medio del Apple Watch con los datos recogidos por la PSG.

Con la señal de frecuencia cardíaca se han decidido sacar 3 vectores de características: mínimo, máximo y media.

### 1.2 Extracción variables de movimiento

### 1.3 Obtención matrices
Realizamos esta operación sobre todos los pacientes:
(TARDA UN POCO, no más de 2 minutos, con movimiento tarda MUCHÍSIMO)

In [4]:
matrix = []
for a, (i,j,k,l) in  enumerate(zip(pacientes_sueño,pacientes_frec_card,pacientes_pasos,pacientes_movimiento)):
    #SUEÑO
    sueño = pd.read_csv(i, delimiter=' ')
    sueño.columns = ["Tiempo","Etiqueta"]
    
    #SUEÑO BINARIO
    sueño_bin = sueño.replace(to_replace =[-1,5],  value =0) 
    sueño_bin = sueño_bin.replace(to_replace =[2,3,4],  value =1)    #Etiquetas binarias
    bin_labels = sueño_bin["Etiqueta"]
    
    #ID_LIST
    labels = sueño["Etiqueta"]
    id_list = [a]*len(labels)
    
    #FRECUENCIA CARDIACA
    frec_card = pd.read_csv(j)
    frec_card.columns = ["Tiempo","Frec card"]
    frec_card=frec_card[frec_card["Tiempo"]>0]
    
    w_size = 30
    signal = frec_card
    n_sueño = len(sueño)
    f_list = get_window(w_size,signal,n_sueño)

    fc_mean_list = []
    fc_min_list = []
    fc_max_list = []

    for f in f_list:
        fc_mean_list.append(round(f['Frec card'].mean(),2))
        fc_min_list.append(round(f['Frec card'].min(),2))
        fc_max_list.append(round(f['Frec card'].max(),2))
    
    feat1 = fc_min_list
    feat2 = fc_max_list
    feat3 = fc_mean_list
    
    #ACCELERACION
    mov=pd.read_csv(l, delimiter=' ')
    mov.columns=["Tiempo","Ac_x","Ac_y","Ac_z"]
    mov_t=mov[mov['Tiempo']>0]
    mov_t=mov_t.sample(frac=.1)
    mov_t=mov_t.sort_values('Tiempo')
    raiz_movto= np.sqrt(mov_t.Ac_x**2 + mov_t.Ac_y**2 + mov_t.Ac_z**2)
    mov_t['Raiz_movto'] = raiz_movto
    signal=mov_t
    n_sueño=len(sueño)
    
    move_list= get_window(w_size,signal,n_sueño)
    mean_acc=[]
    min_acc=[]
    max_acc=[]

    for f in move_list:
        mean_acc.append(round(f['Raiz_movto'].mean(),2))
        min_acc.append(round(f['Raiz_movto'].min(),2))
        max_acc.append(round(f['Raiz_movto'].max(),2))
        
    feat4=min_acc
    feat5=max_acc
    feat6=mean_acc
    
    datos = {'Min_hr': feat1, "Max_hr": feat2,"Mean_hr":feat3,'Min_acc': feat4, "Max_acc": feat5,"Mean_acc":feat6,"Etiquetas":labels,"Etiquetas binarias":bin_labels,"ID_P":id_list}
    datos_paciente = pd.DataFrame(data=datos)
    matrix.append(datos_paciente)

#### FEATURES

* **feat1: min_list**. La primera característica que se extrae es la lista de frecuencias mínimas.
* **feat2: max_list**. La segunda característica que se extrae es la lista de frecuencias máximas.
* **feat3: mean_list**. La tercera característica que se extrae es la lista de frecuencias medias.


* **Queremos añadir el movimiento, la raíz cuadrada de la suma de sus 3 componentes (eje x, y, z)** --> problema velocidad--> ¿paralelizar?


#### OUTCOME

* **labels:**. Lista de etiquetas del sueño

#### IDENTIFICACION
* **ID:**. Variable que identifica a cada paciente.
¿Porqué valores NaN?

In [5]:
#matrix

In [6]:
print(len(matrix))

31


In [7]:
#matrix[0][-5:]

Como hay valores NaN en algunas de las matrices, se utiliza la siguiente funcion para limpiar el dataframe de Nan y valores infinitos:

In [8]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [9]:
clean_matrix = clean_dataset(pd.concat(matrix))

## 2. Conjuntos Train y Test

In [10]:
lista = list(range(31)) # Lista de 31 numeros
pac_train = sample(lista,k=21) # 21 pacientes para train
pac_test = [el for el in lista if el not in pac_train] #Los que no estén en train que vayan a test (10)

print(len(pac_train),'pacientes de train:',pac_train)
print(len(pac_test),'pacientes de test:',pac_test)

21 pacientes de train: [19, 11, 29, 7, 16, 23, 15, 3, 2, 8, 21, 20, 25, 14, 0, 10, 6, 24, 1, 9, 4]
10 pacientes de test: [5, 12, 13, 17, 18, 22, 26, 27, 28, 30]


In [11]:
#print(len(clean_matrix)) #¿¿¿LONGITUD VARIABLE Y LONGITUD DF???? DISTINTOS
#clean_matrix

In [12]:
train_matr = []
test_matr = []
id_paciente = clean_matrix['ID_P']

for i in pac_train:    
    idx = []
    for d in id_paciente:
        if d==i:
            idx.append(True)
        else:
            idx.append(False)
    m = clean_matrix[idx]
    train_matr.append(m)

for i in pac_test:    
    idx = []
    for d in id_paciente:
        if d==i:
            idx.append(True)
        else:
            idx.append(False)
    m = clean_matrix[idx]
    test_matr.append(m)

In [13]:
#train_matr

### 2.1 Preparación datos de entrada y salida

In [14]:
Train = pd.DataFrame()    
for i in range(len(train_matr)):
    x_train=train_matr[i]
    Train=Train.append(x_train)

Test = pd.DataFrame()    
for i in range(len(test_matr)):
    x_test=test_matr[i]
    Test=Test.append(x_test)

In [15]:
Train

,Min_hr,Max_hr,Mean_hr,Min_acc,Max_acc,Mean_acc,Etiquetas,Etiquetas binarias,ID_P
0,72.0,77.0,73.33,0.98,1.02,0.99,-1.0,0.0,19.0
1,70.0,75.0,71.60,0.75,1.39,1.00,-1.0,0.0,19.0
2,71.0,75.0,72.29,0.91,1.05,0.99,-1.0,0.0,19.0
3,70.0,72.0,71.20,0.84,1.13,0.99,-1.0,0.0,19.0
4,68.0,70.0,68.80,0.99,1.00,0.99,-1.0,0.0,19.0
...,...,...,...,...,...,...,...,...,...
982,73.0,74.0,73.40,1.00,1.01,1.00,-1.0,0.0,4.0
983,74.0,76.0,74.80,0.99,1.01,1.00,-1.0,0.0,4.0
984,71.0,76.0,73.33,0.99,1.01,1.00,-1.0,0.0,4.0
985,74.0,77.0,75.71,1.00,1.01,1.00,-1.0,0.0,4.0


In [16]:
Train.to_csv(r'C:\Users\rober\OneDrive\Escritorio\PROCESADO_github\SleepAnalysisWearable\Train.csv')
Test.to_csv(r'C:\Users\rober\OneDrive\Escritorio\PROCESADO_github\SleepAnalysisWearable\Test.csv')

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\rober\\OneDrive\\Escritorio\\PROCESADO_github\\SleepAnalysisWearable'